In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Lambda, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [2]:
train_pairs_w_target = pd.read_parquet(r'C:\Users\pitko\Downloads\train_pairs_w_target.parquet')
train_data = pd.read_parquet(r'C:\Users\pitko\Downloads\train_data.parquet')
test_data = pd.read_parquet(r'C:\Users\pitko\Downloads\test_data.parquet')
test_paires = pd.read_csv(r'C:\Users\pitko\Downloads\test_for_participants.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_data['characteristic_attributes_mapping'] = encoder.fit_transform(train_data['characteristic_attributes_mapping'])
encoder = LabelEncoder()
train_data['name'] = encoder.fit_transform(train_data['name'])
encoder = LabelEncoder()
train_data['categories'] = encoder.fit_transform(train_data['categories'])
encoder = LabelEncoder()
#train_data['main_pic_embeddings_resnet_v1'] = encoder.fit_transform(train_data['main_pic_embeddings_resnet_v1'])

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
test_data['characteristic_attributes_mapping'] = encoder.fit_transform(test_data['characteristic_attributes_mapping'])
encoder = LabelEncoder()
test_data['name'] = encoder.fit_transform(test_data['name'])
encoder = LabelEncoder()
test_data['categories'] = encoder.fit_transform(test_data['categories'])
encoder = LabelEncoder()

In [5]:
train_data

,variantid,characteristic_attributes_mapping,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64
0,51195767,331022,286658,290,[оранжевый],None,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485..."
1,51196903,261830,156905,290,[белый],"[[0.42044494, -0.33794826, -0.037247024, 0.165...","[[0.16211876, -0.4455993, 0.6131705, 0.5954206...","[-0.48503304, 0.6264443, 0.6406273, -0.4696772..."
2,52061880,214890,76201,128,None,"[[0.66729844, -0.023996592, 0.42721167, 0.0205...","[[-0.8638098, -0.1625915, -0.034600798, 0.1337...","[-0.15832633, 0.44736174, 0.31883216, -0.55559..."
3,52067481,160021,76471,128,None,"[[0.3516068, 0.6306597, 0.80962396, -0.0104141...","[[-0.26286322, 0.22858405, 0.4346232, 0.023486...","[-0.42703134, 0.16898727, 0.53922343, -0.53523..."
4,52610752,179586,143013,268,"[черный, чер]","[[-0.009966308, 0.70965785, 0.10790472, -0.411...","[[0.09032486, 0.74205226, 0.30355096, -0.04755...","[-0.4892143, 0.47568643, 0.6505941, -0.4106509..."
...,...,...,...,...,...,...,...,...
457058,822394794,236775,258649,300,"[grey, серый]","[[0.5289667, -0.8587297, 0.6250174, 0.10048234...","[[0.48594046, -0.8139478, -0.11380915, -0.3833...","[-0.56425023, 0.41288334, 0.5501014, -0.330675..."
457059,822409141,289820,301046,339,"[black, черный]","[[-0.25175428, -0.5215634, 0.88485634, 0.72421...","[[0.22031438, -1.3273709, 0.80755866, 0.343482...","[-0.073415995, 0.39561644, 0.4324598, -0.63042..."
457060,822467917,112311,3268,210,"[black, черный, зеленый]",None,"[[0.7038868, -0.2959186, -0.7599652, -0.354495...","[-0.631185, 0.3781632, 0.50562716, -0.65348136..."
457061,822735361,261339,126481,198,[черный],None,"[[0.47696507, 0.568993, -0.024000928, 0.038465...","[-0.47451174, 0.3448143, 0.6183864, -0.5403839..."


import ast

# извлечение столбца
data_col = train_data.characteristic_attributes_mapping

# преобразование строки JSON в словарь
data_list = []
for item in data_col:
    data_dict = ast.literal_eval(item)
    data_list.append(data_dict)

# извлечение значений ключей словаря в список
words_list = []
for data_dict in data_list:
    words_list.append(list(data_dict.values()))

import gensim

from gensim.models import Word2Vec 

# Преобразование списков в текстовый корпус 
text_corpus = []
for words in words_list:    
    text_corpus.append(" ".join(str(word) for word in words))

# Создание корпуса текстов 
corpus = []
for text in text_corpus:     
    corpus.append(text.split())

# Обучение модели word2vec на корпусе текстов 
model = gensim.models.Word2Vec(corpus, min_count=1) 

# Получение числовых векторов для списка слов 
vectors = []
for words in corpus:    
    words_vectors = [] 
    for word in words:       
        try:             
            vector = model.wv.get_vector(word) 
            words_vectors.append(vector)         
        except KeyError:           
            continue     
    vectors.append(words_vectors)
train_data.characteristic_attributes_mapping = vectors

# обработка name

name_list = train_data['name'].tolist()
tokenized_names = [name.split() for name in name_list]
model = Word2Vec(tokenized_names, window=5, min_count=1, workers=4)

name_vectors = []
for name in tokenized_names:
    name_vector = [model.wv[word] for word in name if word in model.wv]
    if name_vector:
        name_mean_vector = np.mean(name_vector, axis=0)
        name_vectors.append(name_mean_vector)
    else:
        name_vectors.append(np.zeros(100))  # Если нет соответствия словам из модели, используйте нулевой вектор

#name_vectors = np.array(name_vectors)
train_data['name'] = name_vectors

# обработка categories

categories_list = train_data["categories"].tolist()
tokenized_categories = [category.split() for category in categories_list]
model = Word2Vec(tokenized_categories, window=5, min_count=1, workers=4)

category_vectors = []
for category in tokenized_categories:
    category_vector = [model.wv[word] for word in category if word in model.wv]
    if category_vector:
        category_mean_vector = np.mean(category_vector, axis=0)
        category_vectors.append(category_mean_vector)
    else:
        category_vectors.append(np.zeros(100))  # Если нет соответствия словам из модели, используйте нулевой вектор

#category_vectors = np.array(category_vectors)
train_data["categories"] = category_vectors

In [6]:
merged_df = train_pairs_w_target.merge(
    train_data, left_on='variantid1', right_on='variantid'
)
merged_df = merged_df.merge(
    train_data, left_on='variantid2', right_on='variantid', suffixes=('_1', '_2')
)

# Выбор необходимых столбцов
columns_1 = [
    'variantid_1', 'characteristic_attributes_mapping_1',
    'name_1', 'categories_1', 'color_parsed_1', 'pic_embeddings_resnet_v1_1',
    'main_pic_embeddings_resnet_v1_1', 'name_bert_64_1'
]
columns_2 = [
    'variantid_2', 'characteristic_attributes_mapping_2',
    'name_2', 'categories_2', 'color_parsed_2', 'pic_embeddings_resnet_v1_2',
    'main_pic_embeddings_resnet_v1_2', 'name_bert_64_2'
]
target_column = ['target']

# Создание нового DataFrame с выбранными столбцами
new_df = merged_df[columns_1 + columns_2 + target_column]

# Вывод нового DataFrame
display(new_df)




,variantid_1,characteristic_attributes_mapping_1,name_1,categories_1,color_parsed_1,pic_embeddings_resnet_v1_1,main_pic_embeddings_resnet_v1_1,name_bert_64_1,variantid_2,characteristic_attributes_mapping_2,name_2,categories_2,color_parsed_2,pic_embeddings_resnet_v1_2,main_pic_embeddings_resnet_v1_2,name_bert_64_2,target
0,51197862,261847,286520,290,[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...",51198054,261845,286518,290,[белый],None,"[[-0.42941108, -0.5129398, -0.4753536, -0.0677...","[-0.455473, 0.58157134, 0.5870387, -0.5325003,...",0.0
1,51197862,261847,286520,290,[белый],None,"[[-0.4304909, -0.49474272, -0.46439183, -0.060...","[-0.5104684, 0.56158644, 0.58873796, -0.529718...",51199884,261846,286519,290,[белый],None,"[[-0.43180764, -0.49580905, -0.5062628, -0.130...","[-0.5425725, 0.6415736, 0.51481575, -0.5687392...",0.0
2,53062686,49520,145331,268,"[черный, чер]",None,"[[-0.0032773763, 0.32531193, -0.33156675, 0.41...","[-0.53706163, 0.37264067, 0.44363657, -0.37289...",536165289,291570,145326,268,[пурпурный],None,"[[-0.043616347, 0.49310583, -0.3069673, 0.4820...","[-0.51572454, 0.40346462, 0.43528882, -0.34104...",1.0
3,53602615,290959,145327,268,[пурпурный],None,"[[0.027616128, 0.33428708, -0.37326592, 0.4108...","[-0.61162275, 0.4953002, 0.47400212, -0.429568...",587809782,51025,145329,268,"[черный, чер]",None,"[[-0.041107245, 0.48485547, -0.2837791, 0.4637...","[-0.5087511, 0.46164495, 0.42914906, -0.462508...",1.0
4,53602615,290959,145327,268,[пурпурный],None,"[[0.027616128, 0.33428708, -0.37326592, 0.4108...","[-0.61162275, 0.4953002, 0.47400212, -0.429568...",615149925,291570,145326,268,[пурпурный],None,"[[-0.043616347, 0.49310583, -0.3069673, 0.4820...","[-0.51572454, 0.40346462, 0.43528882, -0.34104...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306535,810680230,225646,160393,268,[черный],None,"[[-0.0944288, 0.8932959, 0.25614977, -0.191978...","[-0.6026151, 0.3903093, 0.74605453, -0.6013487...",820119986,225643,159182,268,[черный],None,"[[-0.0944288, 0.8932959, 0.25614977, -0.191978...","[-0.5632421, 0.41563767, 0.7506991, -0.6037065...",0.0
306536,812434186,282339,104568,339,[желтый],"[[0.28905904, -2.2125282, -0.51811993, 0.59951...","[[0.17367405, -1.9180702, -0.3562176, 1.175615...","[-0.73866224, 0.50003415, 0.38444322, -0.50629...",815345877,282340,104567,339,[серебристый],"[[0.28905904, -2.2125282, -0.51811993, 0.59951...","[[0.17367405, -1.9180702, -0.3562176, 1.175615...","[-0.7552949, 0.56776017, 0.48786828, -0.515838...",0.0
306537,815637954,315545,130182,203,None,"[[0.025763443, 0.29543704, -0.41198447, 0.1375...","[[0.19128616, 0.530021, 0.57061875, 0.59129846...","[-0.41885406, 0.4686782, 0.48758537, -0.452275...",817550808,315451,130146,203,None,"[[0.104727335, 0.1895248, 0.649516, 0.18494824...","[[0.13195059, 0.55129564, 0.56075263, 0.599945...","[-0.41885418, 0.46867815, 0.48758543, -0.45227...",0.0
306538,817327230,320688,255800,300,[зеленый],"[[-0.15069294, -0.015367113, 0.45699638, -0.61...","[[0.12376598, -0.56147766, 0.8182319, 0.356286...","[-0.6552933, 0.43776116, 0.61501455, -0.564846...",822083612,323924,255792,300,[зеленый],"[[-0.15069294, -0.015367113, 0.45699638, -0.61...","[[0.12376598, -0.56147766, 0.8182319, 0.356286...","[-0.63105303, 0.45106313, 0.63246083, -0.55864...",0.0


In [7]:
merged_df_2 = test_paires.merge(
    test_data, left_on='variantid1', right_on='variantid'
)
merged_df_2 = merged_df_2.merge(
    test_data, left_on='variantid2', right_on='variantid', suffixes=('_1', '_2')
)

# Выбор необходимых столбцов
columns_1 = [
    'variantid_1', 'characteristic_attributes_mapping_1',
    'name_1', 'categories_1', 'color_parsed_1', 'pic_embeddings_resnet_v1_1',
    'main_pic_embeddings_resnet_v1_1', 'name_bert_64_1'
]
columns_2 = [
    'variantid_2', 'characteristic_attributes_mapping_2',
    'name_2', 'categories_2', 'color_parsed_2', 'pic_embeddings_resnet_v1_2',
    'main_pic_embeddings_resnet_v1_2', 'name_bert_64_2'
]


# Создание нового DataFrame с выбранными столбцами
new_df_2 = merged_df_2[columns_1 + columns_2 ]
new_df_2['target'] = 0
# Вывод нового DataFrame
display(new_df_2)



C:\Users\pitko\AppData\Local\Temp\ipykernel_9896\3691931275.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_2['target'] = 0


,variantid_1,characteristic_attributes_mapping_1,name_1,categories_1,color_parsed_1,pic_embeddings_resnet_v1_1,main_pic_embeddings_resnet_v1_1,name_bert_64_1,variantid_2,characteristic_attributes_mapping_2,name_2,categories_2,color_parsed_2,pic_embeddings_resnet_v1_2,main_pic_embeddings_resnet_v1_2,name_bert_64_2,target
0,372065731,10620,30567,156,"[голубой, cyan]",None,"[[0.7827438, 0.36260444, -0.36691827, -0.01188...","[-0.5526009, 0.44057852, 0.5622414, -0.4666865...",372070671,10614,30563,156,"[пурпурный, magenta]",None,"[[0.7827438, 0.36260444, -0.36691827, -0.01188...","[-0.43066338, 0.52844566, 0.53977954, -0.28537...",0
1,528068358,11881,276,106,[черный],"[[0.41624135, -0.2728767, -0.63212967, -0.7030...","[[0.7432036, -0.0027395152, -0.5901706, -0.526...","[-0.7677223, 0.44488266, 0.5248284, -0.4583808...",533445916,1690,274,106,[черный],"[[0.63666844, -0.10665176, -0.9999647, -0.2893...","[[0.7382202, -0.0037149861, -0.58932275, -0.52...","[-0.75576663, 0.376524, 0.48415232, -0.5606043...",0
2,480899084,1341,12122,85,[черный],None,"[[0.2437792, -0.2956852, 1.1284342, 0.7640408,...","[-0.74123424, 0.5190009, 0.48850143, -0.680078...",480899989,1341,12128,85,[черный],None,"[[-0.09049302, -0.19344139, 0.74216104, 0.7289...","[-0.7173982, 0.5979674, 0.59332, -0.6825029, 0...",0
3,384479474,20206,22768,206,[черный],"[[0.01614677, 0.59215486, -0.5727873, 0.391154...","[[0.21851645, 0.7160463, -0.6478431, 0.3415811...","[-0.4883215, 0.61533916, 0.70429194, -0.413429...",488458602,20204,22741,206,"[black, черный]",None,"[[0.20587248, 0.7206383, -0.6515019, 0.3226950...","[-0.20958138, 0.49075866, 0.5546755, -0.506469...",0
4,658044199,1231,11812,179,[черный],None,"[[0.8218225, 0.29885513, 1.6345181, 1.0784931,...","[-0.21744387, 0.29431003, 0.5979793, -0.596469...",663187587,834,9595,179,"[розовый, золотой]",None,"[[0.15543592, 0.12636015, 0.90599126, 0.381110...","[-0.3630666, 0.23649594, 0.5702316, -0.5300651...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18081,405934001,16112,10201,44,[черный],"[[0.0057342947, 0.2522542, -0.14824927, 0.5214...","[[-1.3721861, 0.7138972, -0.1133509, 1.4226291...","[-0.57441187, 0.37751305, 0.3722265, -0.481711...",695957034,16130,10200,44,None,"[[-0.14194776, 0.9729681, -0.36813924, 0.62720...","[[-1.1223259, 0.6643252, -0.18542057, 1.434844...","[-0.5933971, 0.35881573, 0.40324795, -0.564622...",0
18082,692224892,25790,21196,102,"[голубой, белый]","[[-0.19083267, 0.41586366, -0.1883522, 0.11111...","[[-0.7371615, 0.11637456, -0.19845346, 0.04189...","[-0.27254334, 0.43074992, 0.64058787, -0.50700...",692246989,25786,21196,102,"[бежевый, белый]","[[-0.8730906, 0.13971516, 0.34275776, 0.286916...","[[-1.0177655, -0.3487316, 0.5591593, 0.4445994...","[-0.27254334, 0.43074992, 0.64058787, -0.50700...",0
18083,388824213,26933,8654,38,None,"[[-1.1269205, -0.042348184, 0.067774944, 1.160...","[[-0.49422517, -0.195599, -0.32973167, 1.63259...","[-0.395658, 0.59793526, 0.112241246, -0.455735...",658486072,24127,8764,38,"[черный, gold]","[[-0.6395842, -0.09249519, -0.11197661, 1.3431...","[[-0.5299228, 0.17396028, -0.2516922, 0.866433...","[-0.5073685, 0.65060246, 0.041921962, -0.46520...",0
18084,479858327,21339,11334,77,None,None,"[[0.1464705, 0.40828937, 0.40342772, 0.2070388...","[-0.5335841, 0.72697157, 0.5277968, -0.6359731...",480111601,21338,24530,77,None,None,"[[0.1464705, 0.40828937, 0.40342772, 0.2070388...","[-0.50106055, 0.763777, 0.5796494, -0.6116073,...",0


In [8]:
test_data

,variantid,characteristic_attributes_mapping,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64
0,51199146,18110,30996,175,[белый],"[[-0.35485864, -1.0216757, 0.14074168, -0.0557...","[[-0.4282946, -1.0532691, 0.19847812, 0.215870...","[-0.5053151, 0.65614533, 0.54926777, -0.273817..."
1,52077183,15897,7819,35,None,None,"[[-0.37371367, 0.21449876, -0.12548137, 0.4680...","[-0.5038342, 0.39224583, 0.52048385, -0.484975..."
2,53160720,18445,16967,156,"[черный, чер]","[[0.43723586, 0.9522821, 0.4157864, -0.0524228...","[[0.4081133, 1.0809662, 0.59235734, -0.0116817...","[-0.255614, 0.44175097, 0.54067326, -0.3824608..."
3,61116999,24449,32157,217,[серый],"[[0.30746424, -0.108188055, -0.27209604, -0.30...","[[0.3707685, 0.25092605, -0.54618144, -0.49278...","[-0.70480907, 0.30916372, 0.726041, -0.3657149..."
4,65634162,19824,17013,156,"[black, черный]",None,"[[-0.096639805, 0.90385234, 0.16770718, -0.580...","[-0.27215916, 0.38432896, 0.58664465, -0.42248..."
...,...,...,...,...,...,...,...,...
35725,701743917,7076,14795,96,[черный],None,"[[1.0195711, -0.32520157, 1.6646528, -0.435901...","[-0.6647147, 0.47435802, 0.66871643, -0.397985..."
35726,702196564,11030,32209,218,[белый],"[[-0.11473344, 1.034952, 0.34980518, 0.0128045...","[[0.8366248, 0.16651723, -0.41232893, 0.816281...","[-0.5870994, 0.6454741, 0.39559573, -0.5826778..."
35727,702380688,24373,11567,77,[черный],None,"[[0.12927753, -0.62213755, 0.7779633, -0.62430...","[-0.54395807, 0.16947688, 0.52252334, -0.66651..."
35728,703840561,8860,23501,128,None,None,"[[-0.5705995, -0.3861203, -0.053547814, -0.040...","[-0.48934427, 0.5409883, 0.7418469, -0.3841671..."


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Subtract
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Subtract
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

# чтение данных из датафрейма


# разделение данных на train и test
train_data, test_data = train_test_split(new_df, test_size=0.2, random_state=42)

# создание функции для создания пар объектов из данных
def create_pairs(x):
    pairs = []
    labels = []
    for i in range(len(x)-1):
        for j in range(i+1, len(x)):
            pairs.append([x[i][0], x[j][0]])
            if x[i][1] == x[j][1]:
                labels.append(1)
            else:
                labels.append(0)
    return np.array(pairs), np.array(labels)

# создание пар объектов из данных
train_pairs, train_labels = create_pairs(train_data[['main_pic_embeddings_resnet_v1_1', 'target']].values)
test_pairs, test_labels = create_pairs(test_data[['main_pic_embeddings_resnet_v1_1', 'target']].values)

# задание гиперпараметров нейросети
input_shape = (train_pairs.shape[2],)
embedding_dim = 128
dense_dim = 64

# создание модели сиамской нейронной сети
input_1 = Input(input_shape)
input_2 = Input(input_shape)

embedding_layer = Dense(embedding_dim, activation='relu')

embedding_1 = embedding_layer(input_1)
embedding_2 = embedding_layer(input_2)

distance_layer = Lambda(lambda x: K.abs(x[0]-x[1]))

distance = distance_layer([embedding_1, embedding_2])

dense = Dense(dense_dim, activation='relu')(distance)
output = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=[input_1, input_2], outputs=output)

# компиляция модели
rms = RMSprop()
model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])

# обучение модели
history = model.fit([train_pairs[:,0,:], train_pairs[:,1,:]], train_labels,
                batch_size=32,
                epochs=20,
                validation_data=([test_pairs[:,0,:], test_pairs[:,1,:]], test_labels))

# оценка модели
test_loss, test_acc = model.evaluate([test_pairs[:,0,:], test_pairs[:,1,:]], test_labels, verbose=1)
print('Test accuracy:', test_acc)

# разделение данных на train и test
train_data, test_data = train_test_split(new_df, test_size=0.2, random_state=42)

# создание функции для создания пар объектов из данных
def create_pairs(x):
    pairs = []
    labels = []
    for i in range(len(x)-1):
        for j in range(i+1, len(x)):
            pairs.append([x[i][0], x[j][0]])
            if x[i][1] == x[j][1]:
                labels.append(1)
            else:
                labels.append(0)
    return np.array(pairs), np.array(labels)

# создание пар объектов из данных
train_pairs, train_labels = create_pairs(train_data[['main_pic_embeddings_resnet_v1_1', 'target']].values)
test_pairs, test_labels = create_pairs(test_data[['main_pic_embeddings_resnet_v1_1', 'target']].values)

# задание гиперпараметров нейросети
input_shape = (train_pairs.shape[2],)
embedding_dim = 128
dense_dim = 64

# создание модели сиамской нейронной сети
input_1 = Input(input_shape)
input_2 = Input(input_shape)

embedding_layer = Dense(embedding_dim, activation='relu')

embedding_1 = embedding_layer(input_1)
embedding_2 = embedding_layer(input_2)

distance_layer = Lambda(lambda x: K.abs(x[0]-x[1]))
distance = distance_layer([embedding_1, embedding_2])
dense = Dense(dense_dim, activation='relu')(distance)
output = Dense(1, activation='sigmoid')(dense)
model = Model(inputs=[input_1, input_2], outputs=output)
rms = RMSprop()
model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy'])
history = model.fit([train_pairs[:,0,:], train_pairs[:,1,:]], train_labels,
                batch_size=32,
                epochs=20,
                validation_data=([test_pairs[:,0,:], test_pairs[:,1,:]], test_labels))
test_loss, test_acc = model.evaluate([test_pairs[:,0,:], test_pairs[:,1,:]], test_labels, verbose=1)
print('Test accuracy:', test_acc)

new_df['characteristic_attributes_mapping_1'] = new_df['characteristic_attributes_mapping_1'].apply(np.array)

new_df['characteristic_attributes_mapping_1'] = new_df['name_1'].apply(np.array)

new_df['characteristic_attributes_mapping_1'] = new_df['categories_1'].apply(np.array)

new_df['characteristic_attributes_mapping_1'] = new_df['main_pic_embeddings_resnet_v1_1'].apply(np.array)


new_df['characteristic_attributes_mapping_1'] = new_df['characteristic_attributes_mapping_2'].apply(np.array)

new_df['characteristic_attributes_mapping_1'] = new_df['name_2'].apply(np.array)

new_df['characteristic_attributes_mapping_1'] = new_df['categories_2'].apply(np.array)

new_df['characteristic_attributes_mapping_1'] = new_df['main_pic_embeddings_resnet_v1_2'].apply(np.array)

from sklearn.model_selection import train_test_split  

train_data, test_data = train_test_split(new_df, test_size=0.2, random_state=42) 
 
from keras.layers import concatenate  
from keras.layers import Input, Dense, Lambda 
from keras.models import Model  
from keras.optimizers import RMSprop 
import keras.backend as K 
import numpy as np 
 
# задание гиперпараметров нейросети 
input_shape = (4,)  
embedding_dim = 128 
dense_dim = 64  
 
# создание модели сиамской нейронной сети  
input_1_1 = Input(shape=input_shape) 
input_1_2 = Input(shape=input_shape)  
input_1_3 = Input(shape=input_shape) 
input_1_4 = Input(shape=input_shape)  
input_2_1 = Input(shape=input_shape) 
input_2_2 = Input(shape=input_shape)  
input_2_3 = Input(shape=input_shape) 
input_2_4 = Input(shape=input_shape)  
 
embedding_layer = Dense(embedding_dim, activation='relu')  
embedding_1_1 = embedding_layer(input_1_1) 
embedding_1_2 = embedding_layer(input_1_2)  
embedding_1_3 = embedding_layer(input_1_3) 
embedding_1_4 = embedding_layer(input_1_4)  
embedding_2_1 = embedding_layer(input_2_1)  
embedding_2_2 = embedding_layer(input_2_2)  
embedding_2_3 = embedding_layer(input_2_3) 
embedding_2_4 = embedding_layer(input_2_4)  
 
distance_layer = Lambda(lambda x: K.abs(x[0]-x[1]))  
distance_1 = distance_layer([embedding_1_1, embedding_2_1]) 
distance_2 = distance_layer([embedding_1_2, embedding_2_2])  
distance_3 = distance_layer([embedding_1_3, embedding_2_3]) 
distance_4 = distance_layer([embedding_1_4, embedding_2_4])  
 
dense = Dense(dense_dim, activation='relu')  
dense_1 = dense(distance_1) 
dense_2 = dense(distance_2)  
dense_3 = dense(distance_3) 
dense_4 = dense(distance_4)  
 
concatenated = concatenate([dense_1, dense_2, dense_3, dense_4])  
output = Dense(1, activation='sigmoid')(concatenated)  
model = Model(inputs=[input_1_1, input_1_2, input_1_3, input_1_4,   
                      input_2_1, input_2_2, input_2_3, input_2_4],                
              outputs=output)  
rms = RMSprop()  
model.compile(loss='binary_crossentropy', optimizer=rms, metrics=['accuracy']) 
 
# преобразование данных для подачи в модель 
train_pairs = [train_data['characteristic_attributes_mapping_1'].values,   
               train_data['name_1'].values,                 
               train_data['categories_1'].values,   
               train_data['main_pic_embeddings_resnet_v1_1'].values,                 
               train_data['characteristic_attributes_mapping_2'].values,   
               train_data['name_2'].values,                 
               train_data['categories_2'].values,   
               train_data['main_pic_embeddings_resnet_v1_2'].values] 
train_pairs = [np.array(p) for p in train_pairs]  
 
test_pairs = [test_data['characteristic_attributes_mapping_1'].values,   
              test_data['name_1'].values,                
              test_data['categories_1'].values,   
              test_data['main_pic_embeddings_resnet_v1_1'].values,                
              test_data['characteristic_attributes_mapping_2'].values,   
              test_data['name_2'].values,                
              test_data['categories_2'].values,   
              test_data['main_pic_embeddings_resnet_v1_2'].values] 
test_pairs = [np.array(p) for p in test_pairs]  
 
history = model.fit(train_pairs, train_data['target'].values,   
                    batch_size=32,                      
                    epochs=20,   
                    validation_data=(test_pairs, test_data['target'].values))  
test_loss, test_acc = model.evaluate(test_pairs, test_data['target'].values, verbose=1)
print('Test accuracy:', test_acc)

In [32]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

# обучение модели
log_reg.fit(new_df[['characteristic_attributes_mapping_1', 'name_1', 'categories_1',
                   'characteristic_attributes_mapping_2', 'name_2', 'categories_2']], new_df['target'])


# создание тестовых данных


# использование модели для предсказания
biba = log_reg.predict_proba(new_df_2[['characteristic_attributes_mapping_1', 'name_1', 'categories_1', 'characteristic_attributes_mapping_2', 'name_2', 'categories_2']])[:,1]
new_df_2['target'] = biba

C:\Users\pitko\AppData\Local\Temp\ipykernel_9896\650930603.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_2['target'] = biba


In [34]:
apa = new_df_2.target

In [35]:
test_paires['target'] = apa

In [36]:
test_paires

,variantid1,variantid2,target
0,372065731,372070671,0.400095
1,528068358,533445916,0.422871
2,480899084,480899989,0.442011
3,384479474,488458602,0.363026
4,658044199,663187587,0.372762
...,...,...,...
18081,405934001,695957034,0.475051
18082,692224892,692246989,0.438765
18083,388824213,658486072,0.481123
18084,479858327,480111601,0.455954


In [39]:
test_paires.to_csv('test_pairs5.csv', index=False)